Задание 1: Распределённое вычисление среднего значения и стандартного
отклонения

In [1]:
%%writefile main.cpp

#include <mpi.h>      // библиотека MPI
#include <iostream>  // ввод-вывод
#include <vector>    // std::vector
#include <cmath>     // sqrt
#include <cstdlib>   // rand, RAND_MAX

int main(int argc, char* argv[]) {

    // Инициализация среды MPI
    MPI_Init(&argc, &argv);

    int rank, size;

    // Получение номера текущего процесса
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Получение общего количества процессов
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Общий размер массива
    const int N = 1'000'000;

    // Полный массив данных (используется только на процессе rank = 0)
    std::vector<double> data;

    // Массивы для MPI_Scatterv:
    // counts — количество элементов для каждого процесса
    // displs — смещения начала данных для каждого процесса
    std::vector<int> counts(size);
    std::vector<int> displs(size);

    // Процесс с rank = 0 инициализирует массив случайных чисел
    if (rank == 0) {
        data.resize(N);
        for (int i = 0; i < N; i++) {
            data[i] = static_cast<double>(rand()) / RAND_MAX;
        }

        // Вычисление количества элементов для каждого процесса
        int base = N / size; // базовое количество
        int rem  = N % size; // остаток

        for (int i = 0; i < size; i++) {
            // Первые rem процессов получают на один элемент больше
            counts[i] = base + (i < rem ? 1 : 0);

            // Вычисление смещений
            displs[i] = (i == 0) ? 0 : displs[i - 1] + counts[i - 1];
        }
    }

    // Рассылка количества элементов каждому процессу
    int local_n;
    MPI_Scatter(counts.data(), 1, MPI_INT,
                &local_n, 1, MPI_INT,
                0, MPI_COMM_WORLD);

    // Локальный массив данных каждого процесса
    std::vector<double> local_data(local_n);

    // Распределение частей массива между процессами
    MPI_Scatterv(data.data(), counts.data(), displs.data(), MPI_DOUBLE,
                 local_data.data(), local_n, MPI_DOUBLE,
                 0, MPI_COMM_WORLD);

    // Локальные суммы
    double local_sum = 0.0;
    double local_sq_sum = 0.0;

    // Вычисление суммы и суммы квадратов элементов локального массива
    for (int i = 0; i < local_n; i++) {
        local_sum += local_data[i];
        local_sq_sum += local_data[i] * local_data[i];
    }

    // Переменные для глобальных результатов
    double global_sum = 0.0;
    double global_sq_sum = 0.0;

    // Сбор локальных сумм на процессе rank = 0
    MPI_Reduce(&local_sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_sq_sum, &global_sq_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    // Вычисление среднего значения и стандартного отклонения
    if (rank == 0) {
        double mean = global_sum / N;

        // Дисперсия по формуле Var = E[x^2] - (E[x])^2
        double variance = (global_sq_sum / N) - (mean * mean);

        // Стандартное отклонение
        double std_dev = std::sqrt(variance);

        // Вывод результатов
        std::cout << "Mean value: " << mean << std::endl;
        std::cout << "Standard deviation: " << std_dev << std::endl;
    }

    // Завершение работы MPI
    MPI_Finalize();
    return 0;
}


Writing main.cpp


In [3]:
!mpic++ -o main main.cpp
!mpirun --allow-run-as-root --oversubscribe -np 4 ./main

Mean value: 0.500007
Standard deviation: 0.28862


Задание 2: Распределённое решение системы линейных уравнений методом
Гаусса

In [14]:
%%writefile task2.cpp

#include <mpi.h>        // MPI библиотека
#include <iostream>    // ввод-вывод
#include <vector>      // контейнер std::vector
#include <cmath>       // математические функции
#include <algorithm>   // std::min

int main(int argc, char* argv[]) {

    // Инициализация среды MPI
    MPI_Init(&argc, &argv);

    int rank, size;

    // Получение номера текущего процесса
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Получение общего количества процессов
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // ===== Параметры задачи =====
    // Размер системы линейных уравнений (NxN)
    int N = 4; // можно изменять или передавать через аргументы

    // ===== Полная матрица коэффициентов A и вектор правых частей b =====
    // Используются только на процессе rank = 0
    std::vector<double> A;
    std::vector<double> b;

    // Инициализация матрицы и вектора на процессе rank = 0
    if (rank == 0) {
        A.resize(N * N);
        b.resize(N);

        // Заполнение матрицы и вектора значениями
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                // Диагональные элементы больше остальных
                A[i * N + j] = (i == j) ? 4.0 : 1.0;
            }
            b[i] = N + 1;
        }
    }

    // ===== Распределение строк матрицы =====
    // Базовое количество строк на процесс
    int base = N / size;

    // Остаток строк, если N не делится на количество процессов
    int rem  = N % size;

    // Количество строк, обрабатываемых текущим процессом
    int local_rows = base + (rank < rem ? 1 : 0);

    // Смещение глобальных строк для текущего процесса
    int offset = rank * base + std::min(rank, rem);

    // ===== Локальные части матрицы и вектора =====
    std::vector<double> local_A(local_rows * N);
    std::vector<double> local_b(local_rows);

    // ===== Параметры для MPI_Scatterv =====
    std::vector<int> sendcounts_A, displs_A;
    std::vector<int> sendcounts_b, displs_b;

    if (rank == 0) {
        sendcounts_A.resize(size);
        displs_A.resize(size);
        sendcounts_b.resize(size);
        displs_b.resize(size);

        int dispA = 0;
        int dispb = 0;

        // Подсчёт количества элементов и смещений
        for (int i = 0; i < size; i++) {
            int rows = base + (i < rem ? 1 : 0);

            // Для матрицы A
            sendcounts_A[i] = rows * N;
            displs_A[i] = dispA;
            dispA += rows * N;

            // Для вектора b
            sendcounts_b[i] = rows;
            displs_b[i] = dispb;
            dispb += rows;
        }
    }

    // ===== Рассылка матрицы A между процессами =====
    MPI_Scatterv(A.data(), sendcounts_A.data(), displs_A.data(), MPI_DOUBLE,
                 local_A.data(), local_rows * N, MPI_DOUBLE,
                 0, MPI_COMM_WORLD);

    // ===== Рассылка вектора b между процессами =====
    MPI_Scatterv(b.data(), sendcounts_b.data(), displs_b.data(), MPI_DOUBLE,
                 local_b.data(), local_rows, MPI_DOUBLE,
                 0, MPI_COMM_WORLD);

    // ===== Буферы для ведущей строки =====
    std::vector<double> pivot_row(N);
    double pivot_b = 0.0;

    // ===== Прямой ход метода Гаусса =====
    for (int k = 0; k < N; k++) {

        // Определение процесса-владельца ведущей строки k
        int owner;
        if (k < (base + 1) * rem)
            owner = k / (base + 1);
        else
            owner = rem + (k - (base + 1) * rem) / base;

        // Процесс-владелец копирует ведущую строку
        if (rank == owner) {
            int local_k = k - offset;
            for (int j = 0; j < N; j++) {
                pivot_row[j] = local_A[local_k * N + j];
            }
            pivot_b = local_b[local_k];
        }

        // Передача ведущей строки всем процессам
        MPI_Bcast(pivot_row.data(), N, MPI_DOUBLE, owner, MPI_COMM_WORLD);
        MPI_Bcast(&pivot_b, 1, MPI_DOUBLE, owner, MPI_COMM_WORLD);

        // Обнуление элементов ниже ведущего в локальных строках
        for (int i = 0; i < local_rows; i++) {
            int global_i = offset + i;
            if (global_i > k) {
                double factor = local_A[i * N + k] / pivot_row[k];
                for (int j = k; j < N; j++) {
                    local_A[i * N + j] -= factor * pivot_row[j];
                }
                local_b[i] -= factor * pivot_b;
            }
        }
    }

    // ===== Сбор преобразованной матрицы и вектора =====
    MPI_Gatherv(local_A.data(), local_rows * N, MPI_DOUBLE,
                A.data(), sendcounts_A.data(), displs_A.data(), MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    MPI_Gatherv(local_b.data(), local_rows, MPI_DOUBLE,
                b.data(), sendcounts_b.data(), displs_b.data(), MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    // ===== Обратный ход метода Гаусса (только rank = 0) =====
    if (rank == 0) {
        std::vector<double> x(N);

        for (int i = N - 1; i >= 0; i--) {
            x[i] = b[i];
            for (int j = i + 1; j < N; j++) {
                x[i] -= A[i * N + j] * x[j];
            }
            x[i] /= A[i * N + i];
        }

        // Вывод решения системы
        std::cout << "Solution vector x:\n";
        for (int i = 0; i < N; i++) {
            std::cout << "x[" << i << "] = " << x[i] << std::endl;
        }
    }

    // Завершение работы MPI
    MPI_Finalize();
    return 0;
}


Overwriting task2.cpp


In [15]:
!mpic++ -o task2 task2.cpp
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task2

Solution vector x:
x[0] = 0.714286
x[1] = 0.714286
x[2] = 0.714286
x[3] = 0.714286


Задание 3: Параллельный анализ графов (поиск кратчайших путей)

In [25]:
%%writefile task3.cpp

#include <mpi.h>        // MPI библиотека
#include <iostream>    // ввод-вывод
#include <vector>      // std::vector
#include <algorithm>   // std::min
#include <cstdlib>     // rand

// Значение, используемое как "бесконечность"
const double INF = 1e9;

int main(int argc, char* argv[]) {

    // Инициализация среды MPI
    MPI_Init(&argc, &argv);

    int rank, size;

    // Получение номера текущего процесса
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Получение общего количества процессов
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // ===== Параметры задачи =====
    // Размер графа (NxN)
    int N = 6; // можно изменить или передать через аргументы

    // ===== Полная матрица смежности графа =====
    // Создаётся и инициализируется только на процессе rank = 0
    std::vector<double> G;
    if (rank == 0) {
        G.resize(N * N);
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                if (i == j)
                    G[i * N + j] = 0.0;               // расстояние до себя
                else
                    G[i * N + j] = (rand() % 10) + 1; // случайный вес ребра
            }
        }
    }

    // ===== Распределение строк матрицы между процессами =====
    // Базовое количество строк на процесс
    int base = N / size;

    // Остаток строк, если N не делится на size
    int rem  = N % size;

    // Количество строк, обрабатываемых текущим процессом
    int local_rows = base + (rank < rem ? 1 : 0);

    // Смещение глобальных строк для текущего процесса
    int offset = rank * base + std::min(rank, rem);

    // ===== Локальная часть матрицы смежности =====
    std::vector<double> local_G(local_rows * N);

    // ===== Параметры для MPI_Scatterv / MPI_Allgatherv / MPI_Gatherv =====
    // ВАЖНО: эти массивы должны существовать на ВСЕХ процессах
    std::vector<int> sendcounts(size);
    std::vector<int> displs(size);

    int disp = 0;
    for (int i = 0; i < size; i++) {
        int rows = base + (i < rem ? 1 : 0);
        sendcounts[i] = rows * N; // количество элементов
        displs[i] = disp;         // смещение
        disp += rows * N;
    }

    // ===== Распределение строк матрицы между процессами =====
    MPI_Scatterv(
        G.data(),                 // исходная матрица (rank 0)
        sendcounts.data(),        // сколько отправлять
        displs.data(),            // смещения
        MPI_DOUBLE,
        local_G.data(),           // локальный буфер
        local_rows * N,           // размер локального буфера
        MPI_DOUBLE,
        0,
        MPI_COMM_WORLD
    );

    // ===== Буфер для хранения полной матрицы на каждом процессе =====
    // Необходим для MPI_Allgatherv
    std::vector<double> full_G(N * N);

    // ===== Алгоритм Флойда–Уоршелла =====
    for (int k = 0; k < N; k++) {

        // Все процессы получают актуальную версию матрицы
        MPI_Allgatherv(
            local_G.data(),
            local_rows * N,
            MPI_DOUBLE,
            full_G.data(),
            sendcounts.data(),
            displs.data(),
            MPI_DOUBLE,
            MPI_COMM_WORLD
        );

        // Обновление расстояний для локальных строк
        for (int i = 0; i < local_rows; i++) {
            int global_i = offset + i;

            for (int j = 0; j < N; j++) {
                // Проверка пути через промежуточную вершину k
                double new_dist =
                    full_G[global_i * N + k] + full_G[k * N + j];

                // Если путь короче — обновляем
                if (new_dist < local_G[i * N + j]) {
                    local_G[i * N + j] = new_dist;
                }
            }
        }
    }

    // ===== Сбор итоговой матрицы кратчайших путей =====
    MPI_Gatherv(
        local_G.data(),
        local_rows * N,
        MPI_DOUBLE,
        G.data(),
        sendcounts.data(),
        displs.data(),
        MPI_DOUBLE,
        0,
        MPI_COMM_WORLD
    );

    // ===== Вывод результата =====
    if (rank == 0) {
        std::cout << "Shortest paths matrix:\n";
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                std::cout << G[i * N + j] << " ";
            }
            std::cout << std::endl;
        }
    }

    // Завершение работы MPI
    MPI_Finalize();
    return 0;
}


Overwriting task3.cpp


In [26]:
!mpic++ -o task3 task3.cpp
!mpirun --allow-run-as-root --oversubscribe -np 2 ./task3

Shortest paths matrix:
0 4 7 7 6 4 
3 0 6 3 6 2 
3 2 0 1 4 4 
4 1 7 0 3 3 
2 6 7 8 0 3 
1 3 4 5 6 0 
